# Пишем токенайзер

In [1]:
import nltk
import re
import pandas as pd
import numpy as np
from string import punctuation
punctuation = punctuation + '\n'
import requests
from bs4 import BeautifulSoup

In [2]:
import torch

ModuleNotFoundError: No module named 'torch'

In [3]:
from transformers import RobertaForMaskedLM
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./gawikimodel", max_len=512)
model = RobertaForMaskedLM.from_pretrained('./gawikimodel/checkpoint-4000')

D:\ProgramData\anaconda3\envs\wikienv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Text Rank

In [5]:
from nltk.stem import WordNetLemmatizer
import re


In [6]:
def clean_tr(sentence):
  sentence = sentence.lower()
  sentence = re.sub(r'http\S+',' ',sentence)
  sentence = re.sub(r'[^a-zA-Z]',' ',sentence)
  sentence = sentence.split()
  sentence = [lem.lemmatize(word) for word in sentence]
  sentence = ' '.join(sentence)
  return sentence

In [7]:
from nltk.probability import FreqDist
def textrank(text, num_sentences=3):
    # Text into sentences
    sentences = text.split('.')
    
    # Text into words
    prepared_sentences = [clean_tr(sentence) for sentence in sentences]
    words = [sentence.split() for sentence in prepared_sentences]
    words = sum(words, []) #flatten the list
    # calculate word frequencies
    fdist = FreqDist(words)
    
    # Assign scores to sentences based on word frequencies
    sentence_scores = {}
    for i, sentence in enumerate(prepared_sentences):
        for word in sentence.split():
            if word in fdist:
                if i in sentence_scores:
                    sentence_scores[i] += fdist[word]
                else:
                    sentence_scores[i] = fdist[word]
    
    # Sort sentences by scores in descending order
    sorted_sentences = sorted(sentence_scores, key=lambda x: sentence_scores[x], reverse=True)
    
    # Select the top `num_sentences` sentences for the summary
    summary_sentences = sorted(sorted_sentences[:num_sentences])
    
    # Create the summary
    summary = ' '.join([sentences[i] for i in summary_sentences])
    
    return summary

# SumBasic

In [8]:
def clean_sb(sentence):
  sentence = sentence.lower()
  sentence = re.sub(r'http\S+',' ',sentence)
  sentence = re.sub(r'[^a-zA-Z]',' ',sentence)
  sentence = sentence.split()
  sentence = [lem.lemmatize(word) for word in sentence]
  sentence = ' '.join(sentence)
  return sentence

In [9]:
def init_probability(sentences):
    probability_dict = {}
    #words = '. '.join(sentences)
    words = [sentence.split() for sentence in sentences]
    words = sum(words, []) #flatten the list
    total_words = len(set(words))
    for word in words:
        if word!='.':
            if not probability_dict.get(word):
                probability_dict[word] = 1
            else:
                probability_dict[word] += 1
    
    for word,count in probability_dict.items():
        probability_dict[word] = count/total_words 
    
    return probability_dict

In [103]:
def update_probability(probability_dict,word):
    if probability_dict.get(word):
        try:
            probability_dict[word] = probability_dict[word]**2
        except:
            pass
    return probability_dict

In [11]:
def average_sentence_weights(sentences,probability_dict):
	sentence_weights = {}
	for index,sentence in enumerate(sentences):
		if len(sentence) != 0:
			average_proba = sum([probability_dict[word] for word in sentence if word in probability_dict.keys()])
			average_proba /= len(sentence)
			sentence_weights[index] = average_proba 
	return sentence_weights

In [12]:
def generate_summary(sentence_weights,probability_dict,cleaned_article,tokenized_article,summary_length = 30):
    summary = ""
    current_length = 0
    prev_sentence = []
    while current_length < summary_length :
        
        highest_probability_word = max(probability_dict)
        #print(highest_probability_word)
        sentences_with_max_word= [index for index,sentence in enumerate(cleaned_article) if highest_probability_word in sentence.split(' ')]
        sentence_list = sorted([[index,sentence_weights[index]] for index in sentences_with_max_word],key=lambda x:x[1],reverse=True)
        #while ((sentence_list[0][0]) not in prev_sentence):
        summary += cleaned_article[sentence_list[0][0]] + ". "
            #prev_sentence.append(sentence_list[0][0])
            #sentence_list[0].pop(0)
        for word in cleaned_article[sentence_list[0][0]]:
            probability_dict = update_probability(probability_dict,word)
        current_length+=1
    return summary

In [13]:
def sumbasic(article, required_length):
    cleaned_article = []
    for i in article.split('.'):
        cleaned_article.append(clean_sb(i))
    tokenized_article = tokenizer.encode(article)
    #cleaned_article = clean(tokenized_article)
    probability_dict = init_probability(cleaned_article)
    #print(probability_dict.get('b'))
    sentence_weights = average_sentence_weights(cleaned_article,probability_dict)
    summary = generate_summary(sentence_weights,probability_dict,cleaned_article,tokenized_article,required_length)
    return summary

# luhn sum

In [14]:
def clean_lh(article):
	lem = WordNetLemmatizer()
	article =  re.sub(r'\[[^\]]*\]','',article)
	article = article.split('.')
	cleaned_list=[]
	for sent in article:
		sent  = sent.lower()
		word_list = []
		words = sent.split()
		for word in words:
			word_list.append(lem.lemmatize(word.lower()))
		cleaned_list.append(' '.join(word_list))
	return cleaned_list

In [15]:
def get_frequency_dictionary(content):
	frequency = {}
	for sentence in content:
		word_list = sentence.split()#word_tokenize(sentence)
		for word in word_list:
			if word not in [',','.',';','%',')','(','``']:
				if frequency.get(word) is None:
					frequency[word]=1
				else:
					frequency[word]+=1
	return frequency

In [16]:
def get_score(content,frequency_dictionary):
    sentence_score={}
    for sentence in content:
        score=0
        word_list = sentence.split()
        start_idx,end_idx = -1,len(word_list)+1
        index_list=[]
        for word in word_list:
            if word not in [',','.',';','%',')','(','``'] and word in frequency_dictionary.keys():
                index_list.append(word_list.index(word)+1)
            if index_list:
                if max(index_list) != min(index_list):
                    score = len(index_list)**2/(max(index_list) - min(index_list))
                else:
                    score = len(index_list)**2/max(index_list)
        sentence_score[content.index(sentence)] = score
    return sentence_score

In [82]:
def get_summary_luhn(sentence_scores,content,threshold):
    summary = ""
    sentence_indexes = sorted(sentence_scores,key=sentence_scores.get,reverse=True)
    for index in sentence_indexes:
        summary+=content[index]+" " 
    return summary

In [95]:
def luhn(content, word_limit):
    cleaned_content = clean_lh(content)
    threshold = len(cleaned_content)//40
    frequency_dictionary = get_frequency_dictionary(cleaned_content)
    sorted_dictionary = {key: frequency_dictionary[key] for key in sorted(frequency_dictionary,key=frequency_dictionary.get,reverse=True)[:word_limit]}
    sentence_scores = get_score(cleaned_content,sorted_dictionary)
    summary = get_summary_luhn(sentence_scores,cleaned_content,threshold)
    return summary

# Сравнение методов

In [93]:
import json
train_data = []
with open('train.json', 'r', encoding='UTF-8') as file:
    for line in file:
        train_data.append(json.loads(line))
original_list = [train_data[i]['reference'] for i in range(10)]
test_summary_list = [train_data[i]['summary'] for i in range(10)]
length_list = [len(i.split('.')) for i in test_summary_list]

In [35]:
from rouge_score import rouge_scorer
lem = WordNetLemmatizer()
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

In [116]:
rogue1_precision_textrank = []
rogueL_precision_textrank = []
rogue1_recall_textrank = []
rogueL_recall_textrank = []
rogue1_precision_sumbasic = []
rogueL_precision_sumbasic = []
rogueL_recall_sumbasic = []
rogue1_recall_sumbasic = []
rogue1_precision_luhn = []
rogueL_precision_luhn = []
rogue1_recall_luhn = []
rogueL_recall_luhn = []
for i in range(10):
    original = original_list[i]
    test_summary = test_summary_list[i]
    length = length_list[i]
    summary_luhn = luhn(original, length)
    summary_sumbasic = sumbasic(original, length)
    summary_textrank = textrank(original, length)
    scores_luhn = scorer.score(test_summary, summary_luhn)
    print(f'Luhn in text {i}')
    print(scores_luhn)
    scores_sumbasic = scorer.score(test_summary, summary_sumbasic)
    print(f'Sumbasic in text {i}')
    print(scores_sumbasic)
    scores_textrank = scorer.score(test_summary, summary_textrank)
    print(f'Textrank in text {i}')
    print(scores_textrank)
    rogue1_precision_textrank.append(scores_textrank['rouge1'].precision)
    rogueL_precision_textrank.append(scores_textrank['rougeL'].precision)
    rogue1_recall_textrank.append(scores_textrank['rouge1'].recall)
    rogueL_recall_textrank.append(scores_textrank['rougeL'].recall)
    rogue1_precision_sumbasic.append(scores_sumbasic['rouge1'].precision)
    rogueL_precision_sumbasic.append(scores_sumbasic['rougeL'].precision)
    rogue1_recall_sumbasic.append(scores_sumbasic['rouge1'].recall)
    rogueL_recall_sumbasic.append(scores_sumbasic['rougeL'].recall)
    rogue1_precision_luhn.append(scores_luhn['rouge1'].precision)
    rogueL_precision_luhn.append(scores_luhn['rougeL'].precision)
    rogue1_recall_luhn.append(scores_luhn['rouge1'].recall)
    rogueL_recall_luhn.append(scores_luhn['rougeL'].recall)

Luhn in text 0
{'rouge1': Score(precision=0.09705541669206852, recall=0.8943820224719101, fmeasure=0.17510861793983393), 'rougeL': Score(precision=0.047125525818447846, recall=0.43426966292134833, fmeasure=0.08502447340922839)}
Sumbasic in text 0
{'rouge1': Score(precision=0.10363636363636364, recall=0.22415730337078651, fmeasure=0.14174067495559503), 'rougeL': Score(precision=0.08025974025974025, recall=0.17359550561797754, fmeasure=0.10976909413854351)}
Textrank in text 0
{'rouge1': Score(precision=0.11187632793611253, recall=0.8578651685393258, fmeasure=0.19793894614038499), 'rougeL': Score(precision=0.05047988863653015, recall=0.38707865168539324, fmeasure=0.08931233391665046)}
Luhn in text 1
{'rouge1': Score(precision=0.040940465133220416, recall=0.914936914936915, fmeasure=0.07837394972631871), 'rougeL': Score(precision=0.02469540512484292, recall=0.5518925518925519, fmeasure=0.04727538960359794)}
Sumbasic in text 1
{'rouge1': Score(precision=0.1146384479717813, recall=0.29100529